In [1]:
## we need to run this every time
!pip install import-ipynb
import import_ipynb
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=78e0b23060eea442175b2714d35e61322a320415c042ec897e3b4582b26b20e3
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [2]:
'''
L_layered_NN file is imported, which has built in functions
'''
your_module = drive.CreateFile({'id':'170DxGP7AZFSeeyR3U8qpKIEPldNbJUe6'})
your_module.GetContentFile('L_layered_NN.ipynb')
import L_layered_NN as NN


importing Jupyter notebook from L_layered_NN.ipynb


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [242]:
import matplotlib.pyplot as plt

In [7]:
import scipy.io
import pandas as pd
import numpy as np

data = scipy.io.loadmat('/content/drive/My Drive/ASSIGNMENT_2/data5.mat')
data1 = data['x']

In [8]:
### in raw_data each column represent a different sample. There are 73 rows in this, last row represent output labels and remaining 72 rows
### are different features. There are 2148 columns meaning there are 2148 sample points.
raw_data = pd.DataFrame(data1.T)
### shuffling the data
shuffled_raw_data = raw_data.sample(frac = 1, random_state = 7, axis=1).reset_index(drop = True)
shuffled_raw_data.columns = list(range(raw_data.shape[1]))
shuffled_raw_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144,2145,2146,2147
0,9099.953116,18545.804850,31221.136800,6045.378272,11386.552440,2468.029048,40585.112370,10144.683520,41038.504630,31822.221040,19925.187430,14069.625980,16683.410070,22520.057740,41315.970700,6113.170217,24931.096650,25994.024340,38249.789810,23866.593830,16107.264880,28075.543210,7047.109471,6070.891412,1179.567565,36860.232600,31750.789340,22102.103920,65635.577690,12096.310900,5508.673950,20152.433810,17100.588720,43693.261610,31008.997010,35598.746150,52945.225710,45030.438360,93306.034560,25419.605360,...,12533.848830,23178.576160,47975.671260,23477.648680,26171.719110,56006.673620,14437.678950,11996.806540,24213.345070,39760.503330,61682.074970,14833.860000,34429.205460,5000.548119,39572.957130,16617.980730,42572.772250,7566.007893,22534.137970,8707.592580,16632.387970,10325.524880,24823.144510,23283.335880,2370.842746,4357.666674,73693.359590,15838.636600,28946.536940,29974.754790,11955.089390,20224.746840,63761.223360,22276.943330,45450.381960,6796.516312,19811.680390,29283.205920,20215.280930,29142.509520
1,34419.448400,25605.492620,33857.898070,14044.801660,80965.113360,3295.677982,22404.359950,18163.662100,29168.533160,53395.171980,48069.733800,76578.703300,287500.841000,3754.443578,137846.880300,12865.763040,26805.664600,19158.029030,48430.817180,20559.113060,44003.807150,32394.208550,29863.088540,28529.163660,48955.236960,72655.327310,32936.267210,12550.863520,28070.771040,28241.353520,9015.150641,7801.354110,26930.673900,24504.110340,18229.446670,30666.065630,166704.868300,38689.096280,221806.030400,62974.090330,...,19440.199430,27568.521840,127115.418600,15900.476540,40546.016970,62573.914430,16642.457500,278424.182400,4866.320277,8310.973432,16321.713990,50625.023910,39857.276670,24913.796150,80483.075600,28442.626590,8486.314744,73459.153720,49413.721970,28964.725520,19414.312160,20329.308040,19004.234160,16942.109470,5373.603683,25050.588230,31628.537310,4539.945699,58797.840730,13121.141440,41633.102560,8394.099590,61662.241650,57471.372310,172426.543100,21456.627700,9095.749423,25639.635730,22626.776280,27918.902060
2,12591.255480,1668.275131,23264.267890,2527.740590,42101.232220,6675.408924,7666.949916,22084.831070,8404.194211,7228.038964,9851.947712,42135.099340,319101.053600,18858.004360,211014.082200,5630.895972,640.289972,68733.952240,40815.518570,20228.289610,43033.539760,45772.032120,51884.787460,9723.199147,37160.065200,15237.973810,3150.429313,23541.407660,19747.678450,26470.212700,5902.318618,15639.208950,56772.725660,4087.749660,44187.544390,343.908104,237975.910800,5445.026312,149647.966400,27588.450400,...,4459.485119,589.331937,187450.876800,52700.870180,13352.080280,10746.588220,2556.498637,258105.067400,24870.067380,48774.080840,42893.340560,11953.976030,19447.296320,12235.309300,10238.516260,7332.083110,16001.831310,50169.544900,5414.445016,7306.976362,5889.043570,31693.914300,30450.262770,35081.073090,705.997749,14893.790160,33054.896100,13985.761130,9516.354835,8479.313070,20935.185850,16671.576880,9501.053956,70742.957400,44271.891250,7195.320432,10838.061950,3921.840447,38847.549880,1169.222279
3,18620.637120,21654.124980,26745.169910,9413.666310,35655.300420,1212.129428,29570.949680,8628.528354,32991.572690,40799.576310,31530.953590,34795.919750,72326.385480,8417.519040,36873.269790,8084.788568,25707.753240,5427.835545,33137.027910,17133.753100,19313.089240,18782.629410,5493.877003,14870.318230,15696.453870,50934.129770,31560.107560,11442.457380,41921.022070,13579.036180,5789.098745,10069.580950,7798.507501,33141.578740,13567.869690,33048.431450,50391.826670,40508.209340,120133.630800,37306.872510,...,14875.343490,25174.592810,40745.438040,6500.680790,30020.984150,56648.1983

In [9]:
#### separating input data and output labels
X = shuffled_raw_data.iloc[:-1, :]
Y = shuffled_raw_data.iloc[-1:, :].reset_index(drop=True)
print("shape of X is: ", X.shape)
print("shape of Y is: ", Y.shape)

shape of X is:  (72, 2148)
shape of Y is:  (1, 2148)


In [10]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = NN.split(X, Y, val_split=0.1, test_split=0.2)
print("X_train shape is: ", X_train.shape)
print("Y_train shape is: ", Y_train.shape)
print("X_val shape is: ", X_val.shape)
print("Y_val shape is: ", Y_val.shape)
print("X_test shape is: ", X_test.shape)
print("Y_test shape is: ", Y_test.shape)

X_train shape is:  (72, 1503)
Y_train shape is:  (1, 1503)
X_val shape is:  (72, 214)
Y_val shape is:  (1, 214)
X_test shape is:  (72, 431)
Y_test shape is:  (1, 431)


In [11]:
### min and max values of features of X
min_X_train_feat = list(X_train.min(axis=1))
max_X_train_feat = list(X_train.max(axis=1))

In [12]:
### normalising X_train, X_val and X_test
X_train_norm = NN.normalize(X_train, min_X_train_feat, max_X_train_feat)
X_val_norm = NN.normalize(X_val, min_X_train_feat, max_X_train_feat)
X_test_norm = NN.normalize(X_test, min_X_train_feat, max_X_train_feat)

In [26]:
X_train_norm_arr = X_train_norm.to_numpy()
Y_train_arr = Y_train.to_numpy()
X_test_norm_arr = X_test_norm.to_numpy()
Y_test_arr = Y_test.to_numpy()
X_val_norm_arr = X_val_norm.to_numpy()
Y_val_arr = Y_val.to_numpy()

In [14]:
#### sigmoid function
def sigmoid(Z):
  '''
  Z: input of any shape
  returns: sigmoid of input
  '''
  A = 1/(1 + np.exp(-Z))
  return A

In [180]:
def relu(Z):
  Z[Z<0] = 0
  return Z

In [423]:
def model(X, Y, dims, seed, learning_rate = 0.01, epochs = 1000):
  '''
  X and Y: normalised input and output data
  dims: list containing number of neurons for each layer (including input layer)
  seed: to always initialize parameters with same rendom value
  learning_rate: learning_rate of gradient descent
  epochs: number of iterations

  returns:
  parameters: trained parameters
  caches: forward propagation parameters ---- ((A_prev, W, b), (Z)) for every layer
  '''
  costs = []
  parameters = NN.initialize_params(dims, seed)

  for i in range(epochs):
    AL, caches = NN.l_layer_forward(X, parameters)
    cost = np.sum(NN.compute_cost(AL, Y))
    costs.append(cost)

    gradients = NN.l_layer_backward(AL, Y, caches)
    parameters = NN.update_params(parameters, gradients, learning_rate)
  return parameters, caches

In [424]:
'''
Training first autoencoder (input layer, hidden layer, output layer)
number of neurons in each layer is: (input --- X_train.shape[0]
                                     hidden 
                                     output --- X_train.shape[0])
'''
n_input = X_train.shape[0]
n_H1 = 64
autoenc_1_dims = [n_input, n_H1, n_input]
param_autoenc_1, caches_autoenc_1 = model(X_train_norm_arr, X_train_norm_arr, dims = autoenc_1_dims, seed = 8,learning_rate = 0.02, epochs = 1000)

In [425]:
'''
Z1: Hidden layer output from autoencoder 1, before applying activation, because l_layer_forward function in L_layered_NN notebook has relu activation
    for L-1 layers. So for this hidden layer, activation is relu.
    Shape of Z1 ---- (num_hidden_layer_neurons, num_training_examples)
    
    We take Z1 and apply sigmoid activation
    Ouput after applying sigmoid activation should be input of autoencoder 2 
    Same thing should be repeated for autoencoders 2 and 3
'''
Z1 = caches_autoenc_1[0][1]
H1 = sigmoid(Z1)
print("shape of H1 is: ", H1.shape)

shape of H1 is:  (64, 1503)


In [427]:
'''
Training second autoencoder ---- (input_layer, hidden_layer_2, output_layer)
number of neurons in each layer is --- (input_layer
                                        hidden_layer_2
                                        output_layer )
'''
n_H2 = 60
autoenc_2_dims = [n_H1, n_H2, n_H1]
param_autoenc_2, caches_autoenc_2 = model(H1, H1, dims = autoenc_2_dims, seed = 3, learning_rate = 0.05, epochs = 1000)

In [428]:
'''
Z2: Hidden layer output from autoencoder 2 of shape (num_neurons_in_H2, num_training_examples)
    We take Z2 and apply sigmoid activation
    Ouput after applying sigmoid activation should be input of autoencoder 3
'''
Z2 = caches_autoenc_2[0][1]
H2 = sigmoid(Z2)
print("shape of H2 is: ", H2.shape)

shape of H2 is:  (60, 1503)


In [429]:
def one_hot_encoding(Y):
  '''
  Y: output labels

  returns:
  one hot encoding of Y ---- shape (num_samples, num_classes)
  '''
  Y_hot = pd.get_dummies(Y[0]).to_numpy()
  return Y_hot

In [430]:
def elm(X_train, Y_train, num_hidden_neurons, seed):
  '''
  X_train: normalized training data ---- shape is (num_samples, num_features)
  Y_train: one hot encoded Y labels ---- shape is (num_samples, num_classes)
  num_hidden_neurons: number of neurons in hidden layer
  seed: same matrix is initialized everytime

  return:
  V: weight matrix between input and hidden layer ----- shape (num_features, num_hidden_neurons)
  b: bias vector of hidden layer ------ shape (1, num_hidden_neurons)
  W: weight matrix between hidden layer and output layer
  '''
  np.random.seed(seed)
  num_features = X_train.shape[1]                          ## number of features
  V = np.random.randn(num_features, num_hidden_neurons)    ## initializing a matrix
  b = np.random.randn(1, num_hidden_neurons)               ## initializing a bias vector
  hypo = np.dot(X_train, V) + b                            ## hypo = X_train*V + b
  H = np.tanh(hypo)                                        ## tanh activation 
  p_inv = np.linalg.pinv(H)                                ## pseudo-inverse of a matrix
  W = np.dot(p_inv, Y_train)                               ## weight matrix

  return V, b, W

In [431]:
Y_one_hot_train = one_hot_encoding(Y_train.T)
print("shape of Y_one_hot_train is: ",Y_one_hot_train.shape)

shape of Y_one_hot_train is:  (1503, 2)


In [432]:
### training ELM
V3, b3, W3 = elm(H2.T, Y_one_hot_train, num_hidden_neurons = 180, seed = 7)

In [433]:
print("shape of V3 is:", V3.shape)
print("shape of W3 is: ",W3.shape)
print("shape of b3 is: ",b3.shape)

shape of V3 is: (60, 180)
shape of W3 is:  (180, 2)
shape of b3 is:  (1, 180)


In [434]:
### storing parameters in a dictionary
W1 = param_autoenc_1['W1']
b1 = param_autoenc_1['b1']
W2 = param_autoenc_2['W1']
b2 = param_autoenc_2['b1']
new_param = {'W1':W1, 'b1':b1, 'W2':W2, 'b2':b2, 'W3':W3, 'b3':b3}

In [435]:
### predict the labels
def predict(test_data, param, V):
  L = len(new_param)//2
  test_data = np.dot(new_param['W1'], test_data) + new_param['b1']
  test_data = sigmoid(test_data)
  test_data = np.dot(new_param['W2'], test_data) + new_param['b2']
  test_data = sigmoid(test_data)
  test_data = np.dot(test_data.T, V) + param['b'+str(L)]
  H = np.tanh(test_data)
  Yp = np.dot(H, param['W'+str(L)])
  Y_pred = pd.DataFrame(Yp.argmax(axis = 1))                 ### decoding predictions --- (converting to labels like 1,2,...)

  return Y_pred

In [436]:
#### calculating the accuracy
def calculate_accuracy(Y_act, Y_pred):
  tp = tn = fn = fp = 0
  Y_act[0] = Y_act[0].astype(int)
  
  for i in range(len(Y_act)):
    if(Y_act.iloc[i,0] == 0):
      if(Y_pred.iloc[i,0] == 0):
        tp = tp + 1
      else:
        fn = fn + 1

    else:
      if(Y_pred.iloc[i,0] == 0):
        fp = fp + 1
      else:
        tn = tn + 1
  
  accuracy = (tp + tn)/len(Y_act)
  return accuracy

In [438]:
### predicting the training labels and calculating training accuracy
Y_train_predict = predict(X_train_norm_arr, new_param, V3)
acc_train = calculate_accuracy(Y_train.T, Y_train_predict)
print(f"train accuracy is {acc_train * 100} %")

train accuracy is 86.3606121091151 %


In [439]:
### predicting the validation set labels and calculating validation set accuracy
Y_val_predict = predict(X_val_norm_arr, new_param, V3)
acc_val = calculate_accuracy(Y_val.T, Y_val_predict)
print(f"validation accuracy is {acc_val * 100} %")

validation accuracy is 81.30841121495327 %


In [440]:
#### predicting the test set labels and calculating test set accuracy
Y_test_predict = predict(X_test_norm_arr, new_param, V3)
acc_test = calculate_accuracy(Y_test.T, Y_test_predict)
print(f"test accuracy is {acc_test * 100} %")

test accuracy is 81.20649651972158 %
